---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 1 - Creating and Manipulating Graphs

Eight employees at a small company were asked to choose 3 movies that they would most enjoy watching for the upcoming company movie night. These choices are stored in the file `Employee_Movie_Choices.txt`.

A second file, `Employee_Relationships.txt`, has data on the relationships between different coworkers. 

The relationship score has value of `-100` (Enemies) to `+100` (Best Friends). A value of zero means the two employees haven't interacted or are indifferent.

Both files are tab delimited.

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
from networkx.algorithms import bipartite


# This is the set of employees
employees = set(['Pablo',
                 'Lee',
                 'Georgia',
                 'Vincent',
                 'Andy',
                 'Frida',
                 'Joan',
                 'Claude'])

# This is the set of movies
movies = set(['The Shawshank Redemption',
              'Forrest Gump',
              'The Matrix',
              'Anaconda',
              'The Social Network',
              'The Godfather',
              'Monty Python and the Holy Grail',
              'Snakes on a Plane',
              'Kung Fu Panda',
              'The Dark Knight',
              'Mean Girls'])


# you can use the following function to plot graphs
# make sure to comment it out before submitting to the autograder
def plot_graph(G, weight_name=None):
    '''
    G: a networkx G
    weight_name: name of the attribute for plotting edge weights (if G is weighted)
    '''
    %matplotlib notebook
    import matplotlib.pyplot as plt
    
    plt.figure()
    pos = nx.spring_layout(G)
    edges = G.edges()
    weights = None
    
    if weight_name:
        weights = [int(G[u][v][weight_name]) for u,v in edges]
        labels = nx.get_edge_attributes(G,weight_name)
        nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
        nx.draw_networkx(G, pos, edges=edges, width=weights);
    else:
        nx.draw_networkx(G, pos, edges=edges);

### Question 1

Using NetworkX, load in the bipartite graph from `Employee_Movie_Choices.txt` and return that graph.

*This function should return a networkx graph with 19 nodes and 24 edges*

In [2]:
def answer_one():
        
    # read data
    employees_df = pd.read_csv('Employee_Movie_Choices.txt', sep='\t')
    relations_df = pd.read_csv('Employee_Relationships.txt', sep = '\t', header=None)
    
    # create bipartite graph
    B = nx.Graph()
    B.add_nodes_from(employees, bipartite=0) 
    B.add_nodes_from(movies, bipartite=1)
    B.add_edges_from(zip(list(employees_df['#Employee']), list(employees_df['Movie'])))
    
    # plot bipartite graph
#     plot_graph(B)

    # check the results of bipartote graph
    top3_movies = employees_df.groupby('#Employee')['Movie'].apply(', '.join).reset_index()
    
    # list all nodes and edges from bipartite graph
    n_edges = B.number_of_edges()
    n_nodes = B.number_of_nodes()
#     print('Bipartite graph has %d nodes and %d edges.' % (n_nodes, n_edges))   
    
    return B

### Question 2

Using the graph from the previous question, add nodes attributes named `'type'` where movies have the value `'movie'` and employees have the value `'employee'` and return that graph.

*This function should return a networkx graph with node attributes `{'type': 'movie'}` or `{'type': 'employee'}`*

In [3]:
def answer_two():
    
    # read data
    employees_df = pd.read_csv('Employee_Movie_Choices.txt', sep='\t')
    relations_df = pd.read_csv('Employee_Relationships.txt', sep = '\t', header=None)
    
    # create bipartite graph
    B = nx.Graph()
    B.add_nodes_from(employees, bipartite=0, type='employee') 
    B.add_nodes_from(movies, bipartite=1, type='movie')
    B.add_edges_from(zip(list(employees_df['#Employee']), list(employees_df['Movie'])))
    
    # plot bipartite graph
#     plot_graph(B)

    # print nodes of bipartite graph with attributes
#     print('Node attributes of bipartite graph: ', B.nodes(data=True))

    return B

### Question 3

Find a weighted projection of the graph from `answer_two` which tells us how many movies different pairs of employees have in common.

*This function should return a weighted projected graph.*

In [4]:
def answer_three():
        
    # read data
    employees_df = pd.read_csv('Employee_Movie_Choices.txt', sep='\t')
    relations_df = pd.read_csv('Employee_Relationships.txt', sep = '\t', header=None)

    # create bipartite graph
    B = nx.Graph()
    B.add_nodes_from(employees, bipartite=0, type='employee') 
    B.add_nodes_from(movies, bipartite=1, type='movie')
    B.add_edges_from(zip(list(employees_df['#Employee']), list(employees_df['Movie'])))

    # create weighted projected graph
    P = bipartite.weighted_projected_graph(B, employees, ratio=False)

    # plot bipartite graph
#     plot_graph(P)

    # print weights
#     print('Weights for weighted projected bipartite graph: ', P.edges(data=True))
    
    return P

### Question 4

Suppose you'd like to find out if people that have a high relationship score also like the same types of movies.

Find the Pearson correlation ( using `DataFrame.corr()` ) between employee relationship scores and the number of movies they have in common. If two employees have no movies in common it should be treated as a 0, not a missing value, and should be included in the correlation calculation.

*This function should return a float.*

In [64]:
def answer_four():
        
    # read data
    employees_df = pd.read_csv('Employee_Movie_Choices.txt', sep='\t')
    relations_df = pd.read_csv('Employee_Relationships.txt', sep = '\t', header=None)

    # create bipartite graph
    B = nx.Graph()
    B.add_nodes_from(employees, bipartite=0, type='employee') 
    B.add_nodes_from(movies, bipartite=1, type='movie')
    B.add_edges_from(zip(list(employees_df['#Employee']), list(employees_df['Movie'])))

    # create weighted projected graph
    P = bipartite.weighted_projected_graph(B, employees, ratio=False)

    # weights
    no_movies_in_common = P.edges(data=True)

    # data manipulation
    # movies in common
    no_movies_in_common_df = pd.DataFrame(no_movies_in_common)
    no_movies_in_common_df['weight'] = no_movies_in_common_df[2].apply(lambda x: x.get('weight'))
    no_movies_in_common_df = no_movies_in_common_df[[0, 1, 'weight']]

    # merge with relationship score
    relationship_score1 = pd.merge(relations_df,
                                 no_movies_in_common_df[[0, 1, 'weight']],
                                 how = 'left',
                                 left_on = [0, 1],
                                 right_on = [0, 1]
                                  )

    relationship_score2 = pd.merge(relations_df,
                                 no_movies_in_common_df[[0, 1, 'weight']],
                                 how = 'left',
                                 left_on = [0, 1],
                                 right_on = [1, 0]
                                  )

    relationship_score = pd.merge(relationship_score1,
                                 relationship_score2[['key_0', 1, 'weight']],
                                  how = 'left',
                                  left_on = [0, 1],
                                  right_on = ['key_0', 1]
                                 )

    relationship_score['weight'] = (
        np.where(relationship_score['weight_x'].isnull() & relationship_score['weight_y'].notnull(), relationship_score['weight_y'],
            (np.where(relationship_score['weight_x'].notnull() & relationship_score['weight_y'].isnull(), relationship_score['weight_x'], 0)
            )
                )
    )

    # rename columns
    relationship_score = relationship_score[[0, 1, 2, 'weight']]
    relationship_score.columns = ['employee1', 'employee2', 'score', 'movies_in_common']

    # calculate Pearson correlation coefficient
    pearson_corr_coeff = relationship_score['score'].corr(relationship_score['movies_in_common'])

    # with significance
#     from scipy.stats import pearsonr
#     corr_coeff = pearsonr(relationship_score['score'], relationship_score['movies_in_common'])
#     corr_coeff_value = corr_coeff[0]
#     corr_coeff_pvalue = corr_coeff[1]
#     print('Pearson correlation coefficient: %0.6f and its p-value: %0.6f.' % (corr_coeff_value, corr_coeff_pvalue))
    
    return pearson_corr_coeff